In [2]:
from math import floor
from statistics import mean

In [3]:
def estimate_stint_length(average_lap_time, average_fuel_consumption):
    race_length = 21600 # s
    long_stops = 3
    base_pitstop_loss = 20  # Remember that last stint won't have a pitstop!

    fuel_tank_size = 100 # L
    refuel_rate = 0.353  # sec / L
    tire_swap_time = 30

    long_stop_time = long_stops * 180
    effective_race_length = race_length - long_stop_time
    pitstop_length = base_pitstop_loss + max(tire_swap_time, refuel_rate * fuel_tank_size)

    stint_length = average_lap_time * floor(fuel_tank_size / average_fuel_consumption) + pitstop_length  # TODO: Parameter for laps out (tire strategy)

    stint_total = (effective_race_length + average_lap_time - pitstop_length) / stint_length  # Adding average_lap_time to numerator accounts for 6 hours + 1 lap
    # Subtracting pitstop length accounts for the fact that the last stint will not have a pitstop

    return stint_total, stint_length



In [4]:
car_1 = [(93, 2.55), (92, 2.7), (90.5, 2.65)]  # Rake, Dadeler
car_2 = [(92, 2.7), (93.4, 2.61), (91.7, 2.6)]  # Dadeler, Marcus, Marv
car_2_mod = car_2[1:]

print(f"Car 1 stints: {mean([estimate_stint_length(*x)[0] for x in car_1])}")
print(f"Car 2 stints: {mean([estimate_stint_length(*x)[0] for x in car_2])}")
print(f"Car 2 stints without dadeler: {mean([estimate_stint_length(*x)[0] for x in car_2_mod])}")

Car 1 stints: 6.008526530400015
Car 2 stints: 5.970472025708266
Car 2 stints without dadeler: 5.90643506426124
